## Imports

In [1]:
%run main_module_file.ipynb

Módulo principal e bibliotecas importados


## Lendo arquivo das waveforms
#### O arquivo original já foi parcialmente modificado em relação a linhas e colunas

In [2]:
%%time
'''Lendo arquivo das waveforms'''

waveform = pd.read_csv('data/5555_eventos-edit.csv',     index_col = 0) # importa como waveform vs sample

FileNotFoundError: [Errno 2] No such file or directory: 'data/5555_eventos-edit.csv'

## 3. ANÁLISE DE DADOS

## 3.1. Copia o data frame e seleciona a janela de busca

Cria uma cópia das waveforms para poder transladar o base line sem modificar os dados originais;

Define a janela onde procuraremos pelos valores do contorno do pulso

In [ ]:
df = waveform.copy(deep = True)
df = df.iloc[ : , :df.shape[1] - 1 ]

VA_1 = 20 # VA_1 e VA_2 são variáveis arbitrárias para definir o majorante da largura dos pulsos
VA_2 = 70

convert_to_microsec = 1/250

In [ ]:
peaks, _ = find_peaks(-df.event_75, height=40)
peaks

In [ ]:
plt.plot(-df.event_75)

## 3.2. Translação da base-line

É interessante transladar os pontos e fixar a base line no 0. Podemos fazer isso somando ~= -50 em cada um dos valores de toda a coleção de waveforms. Isso servirá para evitar problemas de 'height' no peak_finder()

Análise estatística do base-line:
No caso, tomaremos todos os dados antes do primeiro pulso de cada waveform e consideraremos o base-line como o mesmo para todos os waveforms

        Método:

Para encontrar a base line, selecionaremos a parte mais à esquerda, antes do pulso. Esse pedaço é definido como fora de um valor arbitrário em torno do primeiro pico da waveform, que acaba não sofrendo com a questão da base line. Definimos desse jeito porque a função para calcular o contorno do pulso, além de demorar para ser computada, sofre com as questões do peak_finder acabar encontrando apenas um elemento


In [ ]:
%%time

height = 50 # esse parâmetro aqui é somente para efeito de encontrar os elementos da  base line a priori

baseLines = baseLine(df=df, height=height, VA_1=VA_1)

plt.plot(-df['event_100'])

        Relocação do base-line:
Colocando o valor médio do base line como o parâmetro height do peak-finder, surpreendentemente temos um resultado que não incorre em erros na hora de achar os dois picos de cada waveform. No caso, ele acaba por achar dois picos (não mais nem menos) de maneira mais fácil e exata

In [ ]:
'''
Alterando os valores e recolocando na base line ~= 0
'''

df.iloc[ :  , : df.shape[1] ] -= baseLines.mean() 
# soma ~= -50 em todo o df, exceto pela última coluna 'time'
#df.head(2)

height = baseLines.mean()

plt.plot(-df['event_100'])

In [ ]:
plt.plot(-df['event_1234'].iloc[80:150])

In [ ]:
# %%time
# '''
# Filtrando os dados de acordo com o valor do tempo de vida do muon
# '''
# df_filt_delta_t = filtra_delta_t(df=df, convert_to_microsec=convert_to_microsec, time_in_ADCch=5, height=height)

In [ ]:
# from scipy.fft import fft
# y = fft(df['event_77'].iloc[70:110].to_numpy())
# plt.plot( range(len(y)) , y )
# plt.plot(df['event_77'].iloc[70:110].to_numpy())

## Curve fit exponencial da diferença de tempo

In [ ]:
coeff, fig, center_of_bins, values_histogram = curve_fit_exponencial(df=df, height=height, convert_to_microsec=convert_to_microsec, custom_number_of_bins=100, plot_graph=1, plt_points=1, plt_bars=0)
print(coeff)

In [ ]:
# %%time
# aux1 = []
# aux2 = []
# for i in range(5,1010,250):
#     table = curve_fit_exponencial(df=df , height=height, convert_to_microsec=convert_to_microsec, plt_points=1, plt_bars=1, custom_number_of_bins=i, plot_graph=0 )
#     aux1.append(i) , aux2.append(table['valor']['A'])

# aux_df = pd.DataFrame([aux1,aux2]).T
# aux_df.columns = ['number_bins' , 'A']
# print(aux_df)

# #aux_df.to_csv('data/A_number-of-bins.csv')

# plt.scatter( aux_df['number_bins'] , aux_df['A'] , color='dimgrey' )
# plt.xlabel('Número de bins')
# plt.ylabel('Valor da constante A')
# plt.show()

# def curve_fit_aux(t,a):
#     return(a*1/t)
# opt_coeff, cov = curve_fit(curve_fit_aux, xdata=aux_df['number_bins'], ydata=aux_df['A'])
# print(f'y = {opt_coeff[0]}/x')
# print(cov)

# x = np.linspace(4,1000, 10000)
# plt.plot( 
#     x, curve_fit_aux(x,opt_coeff[0]), color = 'orange' 
#         )
# plt.scatter( aux_df['number_bins'] , aux_df['A'] , color='dimgrey' )
# plt.xlabel('Número de bins')
# plt.ylabel('Valor da constante A')
# plt.show()

In [ ]:
def f_esperada_no_bin(center_of_bins, N_0=5555, gamma=1/2.1969811): #micro_seconds
    # number_of_bins = len(center_of_bins)
    delta_t = center_of_bins[1] - center_of_bins[0]
    a_value = center_of_bins - delta_t/2
    b_value = center_of_bins + delta_t/2
    return(     N_0 * ( np.e**(-gamma*a_value) - np.e**(-gamma*b_value) )     )

f_esperada  = f_esperada_no_bin(center_of_bins)
f_observada = values_histogram

In [ ]:
chi_2, p_valor = chisquare(f_obs=f_observada, f_exp=f_esperada) #rejeitar se p_valor <= alpha = 0.05
chi_2

In [ ]:
plt.scatter(  range(len(f_esperada))  , f_esperada   )
plt.scatter(  range(len(f_observada)) , f_observada  )

In [ ]:
number_of_bins_values = []
chi_2_values          = []
p_valor_values        = []
A_values              = []
for i in range(4,200,10):
    table, center_of_bins, values_histogram = curve_fit_exponencial(df=df , height=height, convert_to_microsec=convert_to_microsec, custom_number_of_bins=i, plot_graph=0 )
    f_esperada  = f_esperada_no_bin(center_of_bins=center_of_bins)
    f_observada = values_histogram
    chi_2, p_valor = chisquare(f_obs=f_observada, f_exp=f_esperada) #rejeitar se p_valor <= alpha = 0.05
    number_of_bins_values.append(i)
    chi_2_values.append(chi_2)
    p_valor_values.append(p_valor)
    A_values.append(table['valor']['A'])

In [ ]:
[ (p_valor_values[i]<=0.05) for i in range(len(p_valor_values)) ]

In [ ]:
p_valor_values

In [ ]:
x = [99.8 , 299.7 , 599.9 , 799.7 , 1300.0 , 1600.0 , 1899.8] #V
y = [0.042 , 0.133 , 0.272 , 0.423 , 0.859 , 1.122 , 1.385] #mA

z = [299.7 , 599.9 , 799.7 , 1300, 1400]
k = [0.133 , 0.272 , 0.423 , 0.858, 0.946]

def fit(x,a,b):
    return(a*x + b)
opt_coeff, opt_cov = curve_fit(f=fit, xdata=x, ydata=y)
plt.scatter(x,y)
X = np.linspace(0,2000,10000)
plt.plot( X , fit(X, opt_coeff[0], opt_coeff[1]) )

## 4.1. Loop para construção e check do peak_finder
Esse loop cria e checa se existem erros na obtenção dos picos através da função peak-finder, quando peaks != 2

In [ ]:
%%time
peaks_xy_0   ,   peaks_xy_1    =    peaks_divididos_01(df=df, height = baseLines.mean())
peaks_xy_0.index = peaks_xy_0.index // 2
peaks_xy_1.index = peaks_xy_1.index // 2

In [ ]:
peaks_xy_0.y.value_counts().sort_index()

In [ ]:
peaks_xy_1.y.value_counts().sort_index()

In [ ]:
height

# 5. Plot espectro de amplitude

## 5.1. Plot do espectro de amplitude do primeiro pulso 

In [ ]:
# sns.histplot( -1*peaks_xy_0['y'].round() )
# plt.title('Espectro de amplitude do muon')
# plt.xlabel('Máximo do pulso')
# plt.savefig("images/espectro_amplitude-pulso_0.png", dpi = 150)

In [ ]:
#sns_plot = sns.histplot( -1*peaks_xy_0.query('y > y.min()')['y'] )
#plt.title('Espectro de amplitude do muon [sem saturação]')
#plt.savefig("images/espectro_amplitude_0_ZOOM.png", dpi = 150)

## 5.3. Plot do espectro de amplitude do segundo pulso 

In [ ]:
# sns.histplot( -1*peaks_xy_1['y'].round() )
# plt.title('Espectro de amplitude do eletron')
# plt.savefig("images/espectro_amplitude_1.png", dpi = 150)

In [ ]:
#sns.histplot( -1*peaks_xy_1.query('y > y.min()')['y'] )
#plt.title('Espectro de amplitude do eletron [sem saturação]')
#plt.savefig("images/espectro_amplitude_1_ZOOM.png", dpi = 150)

# 6. Plot dos espectro de carga

## 6.2. Integração do pulso 

Integramos o pulso para obter a carga gerada com ele. Utilizamos a integração pelo método de Simpson.

In [ ]:
%%time
_ = contorno_limite_arbitrario_picos(
    df=df, peaks_01=(peaks_xy_0,peaks_xy_1), VA_1=VA_1, VA_2=VA_2, height=height
                                    )

In [ ]:
#plt.plot(df.event_6.iloc[2200:2500])
#plt.title('exemplo de problema com os picos')

In [ ]:
%%time
integrais_0 = integral_simples_dataframe(_[0], dx=1)
integrais_1 = integral_simples_dataframe(_[1], dx=1)

In [ ]:
# sns.histplot(-1*integrais_0)
# plt.title("Espectro de carga - pulso do muon (0)")
# plt.xlabel('Carga (unidades)')
# plt.savefig("images/espectro_carga-pulso_0.png", dpi = 150)

In [ ]:
# sns.histplot(-1*integrais_1)
# plt.title("Espectro de carga - pulso do eletron (1)")
# plt.xlabel('Carga (unidades)')
# plt.savefig("images/espectro_carga-pulso_1.png", dpi = 150)

In [ ]:
fig = plt.figure(dpi=200, figsize=(10,8))
fig.suptitle('Espectros de amplitude e carga', fontsize=15)

ax1 = plt.subplot(221)
sns.histplot( -1*peaks_xy_0['y'], color='black' )
plt.xlabel('Pico do pulso (unidades)')
plt.ylabel(r'Contagem / tensão ($unidades^{-1}$)')
plt.title(r"Espectro de amplitude - $\mu$")

ax2 = plt.subplot(222, sharex=ax1, sharey=ax1)
sns.histplot( -1*peaks_xy_1['y'] , color='gray')
plt.xlabel('Pico do pulso (unidades)')
plt.ylabel(r'Contagem / tensão ($unidades^{-1}$)')
plt.title(r"Espectro de amplitude - $e^{-}$")

ax3 = plt.subplot(223)
sns.histplot(-1*integrais_0, color='black')
plt.xlabel('Carga elétrica (unidades)')
plt.ylabel(r'Contagem / carga elétrica ($unidades^{-1}$)')
plt.title(r"Espectro de carga - $\mu$")

ax4 = plt.subplot(224, sharex=ax3, sharey=ax3)
sns.histplot(-1*integrais_1, color='gray')
plt.xlabel('Carga elétrica (unidades)')
plt.ylabel(r'Contagem / carga elétrica ($unidades^{-1}$)')
plt.title(r"Espectro de carga - $e^{-}$")

fig.tight_layout()

plt.savefig('images/espectros_orig.png')

## Filtrando as saturações

In [ ]:
# plt.plot(df.event_4, color = 'green', label = 'event_4')
# plt.plot(df.event_6, color = 'black', label = 'event_6')
# plt.legend()
# plt.title('Saturações em apenas um pico')

In [ ]:
#%%time
#df_filt_sat_tot = filtra_saturacao_total( df=df , height=height , VA_1=VA_1 , VA_2=VA_2 )

In [ ]:
%%time
df_filt_parcial = filtra_saturacao_parcial(df=df, height=height, VA_1=VA_1, VA_2=VA_2)
peaks_xy_0_filt   ,   peaks_xy_1_filt    =    df_filt_parcial
peaks_xy_0_filt.index = peaks_xy_0_filt.index // 2
peaks_xy_1_filt.index = peaks_xy_1_filt.index // 2

In [ ]:
%%time
_ = contorno_limite_arbitrario_picos(df=df, peaks_01=df_filt_parcial, VA_1=VA_1, VA_2=VA_2, height=height)

In [ ]:
%%time
integrais_0_filt = integral_simples_dataframe(_[0], dx=1)
integrais_1_filt = integral_simples_dataframe(_[1], dx=1)

In [ ]:
# sns.histplot( -1*peaks_xy_0_filt['y'].round() )
# plt.title('Espectro de amplitude do muon Filtrado')
# plt.xlabel('Máximo do pulso')
# plt.savefig("images/espectro_amplitude-pulso_0_filtrado.png", dpi = 150)

In [ ]:
# sns.histplot( -1*peaks_xy_1_filt['y'].round() )
# plt.title('Espectro de amplitude do eletron Filtrado')
# plt.savefig("images/espectro_amplitude_1_filtrado.png", dpi = 150)

In [ ]:
# sns.histplot(-1*integrais_0_filt)
# plt.title("Espectro de carga - pulso do muon (0) Filtrado")
# plt.xlabel('Carga (unidades)')
# plt.savefig("images/espectro_carga-pulso_0_filtrado.png", dpi = 150)

In [ ]:
# sns.histplot(-1*integrais_1_filt)
# plt.title("Espectro de carga - pulso do eletron (1) Filtrado")
# plt.xlabel('Carga (unidades)')
# plt.savefig("images/espectro_carga-pulso_1_filtrado.png", dpi = 150)

In [ ]:
fig = plt.figure(dpi=200, figsize=(10,8))
fig.suptitle('Espectros de amplitude e carga filtrados pela saturação', fontsize=15)

ax1 = plt.subplot(221)
sns.histplot( -1*peaks_xy_0_filt['y'], color='black' )
plt.xlabel('Pico do pulso (unidades)')
plt.ylabel(r'Contagem / tensão ($unidades^{-1}$)')
plt.title(r"Espectro de amplitude - $\mu$")

ax2 = plt.subplot(222, sharex=ax1, sharey=ax1)
sns.histplot( -1*peaks_xy_1_filt['y'] , color='gray')
plt.xlabel('Pico do pulso (unidades)')
plt.ylabel(r'Contagem / tensão ($unidades^{-1}$)')
plt.title(r"Espectro de amplitude - $e^{-}$")

ax3 = plt.subplot(223)
sns.histplot(-1*integrais_0_filt, color='black')
plt.xlabel('Carga elétrica (unidades)')
plt.ylabel(r'Contagem / carga elétrica ($unidades^{-1}$)')
plt.title(r"Espectro de carga - $\mu$")

ax4 = plt.subplot(224, sharex=ax3, sharey=ax3)
sns.histplot(-1*integrais_1_filt, color='gray')
plt.xlabel('Carga elétrica (unidades)')
plt.ylabel(r'Contagem / carga elétrica ($unidades^{-1}$)')
plt.title(r"Espectro de carga - $e^{-}$")

fig.tight_layout()

plt.savefig('images/espectros_orig_filt.png')

    Como pedido, um DataFrame que une os valores dos delta_t, picos e integrais


In [ ]:
'''
Como pedido, um DataFrame que une os valores dos delta_t, picos e integrais
'''

_ = pd.DataFrame(   peaks_em_x( df=df, height=height),  columns = ['peak_0', 'peak_1']   )   
delta_x = (   _['peak_1']  -  _['peak_0']   )
delta_x.name = 'delta_x'

_ = pd.DataFrame( [delta_x , -1*integrais_0 , -1*integrais_1] ).T

df_bruto = pd.concat( 
    (_ , peaks_xy_0.reset_index().iloc[: , 1:], peaks_xy_1.reset_index().iloc[: , 1:]) , axis=1
                    )
df_bruto.columns = ['delta_x' , 'integrais_muon' , 'integrais_eletron', 'x_pico_muon' , 'y_pico_muon', 'x_pico_eletron' , 'y_pico_eletron']

df_bruto.to_csv('data/df_resultados_brutos.csv')

In [ ]:
_ = df_bruto['delta_x'].value_counts().sort_index()
plt.scatter(_.index , _)
_

In [ ]:
plt.plot(  df['event_70'].index*convert_to_microsec, df['event_70'].values, color='dimgrey'  )
plt.xlabel(r'Tempo ($\mu$s)')
plt.ylabel('Tensão (chADC)')
plt.title('Exemplo de waveform do decaimento do múon')

In [ ]:
df['event_70'].values